In [1]:
%cd ..

/home/jovyan/working


In [2]:
import math
import os

import pandas as pd
import torch

In [3]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [4]:
from omegaconf import OmegaConf

omega_conf_dir = "./config"

c_main = OmegaConf.load(os.path.join(omega_conf_dir, "main.yaml"))
c_data = OmegaConf.load(os.path.join(omega_conf_dir, "data.yaml"))
c = OmegaConf.merge(c_main, c_data)

c.wandb.enabled = False

In [19]:
from src.ice_cube.data_loader import (
    collate_fn,
    make_dataloader_batch,
)
from src.ice_cube.model import load_pretrained_model
from src.ice_cube.scoring import angular_dist_score
from src.ice_cube.submission import to_submission_df

## Dataloader

In [15]:
meta_df = pd.read_parquet("../input/icecube-neutrinos-in-deep-ice/train_meta.parquet")

In [16]:
sensor_df = pd.read_csv("../input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv")

In [21]:
batch_df = pd.read_parquet("../input/icecube-neutrinos-in-deep-ice/train/batch_391.parquet").reset_index()

In [25]:
batch_df = pd.merge(batch_df, sensor_df, on="sensor_id").sort_values("event_id")

In [30]:
dataloader = make_dataloader_batch(c, meta_df, batch_df, collate_fn)

## Predict

In [27]:
model = load_pretrained_model(c, dataloader, state_dict_path=c.inference_params.model_path)

2023-04-04 03:51:53,446 [INFO] [model] Load model from: dynedge-pretrained/dynedge_pretrained_batch_1_to_50/state_dict.pth


In [28]:
results = model.predict(
    gpus=[0],
    dataloader=dataloader,
)

2023-04-04 03:51:54,938 [INFO] [rank_zero] GPU available: True (cuda), used: True
2023-04-04 03:51:54,939 [INFO] [rank_zero] TPU available: False, using: 0 TPU cores
2023-04-04 03:51:54,941 [INFO] [rank_zero] IPU available: False, using: 0 IPUs
2023-04-04 03:51:54,942 [INFO] [rank_zero] HPU available: False, using: 0 HPUs
2023-04-04 03:51:54,948 [INFO] [rank_zero] GPU available: True (cuda), used: True
2023-04-04 03:51:54,949 [INFO] [rank_zero] TPU available: False, using: 0 TPU cores
2023-04-04 03:51:54,950 [INFO] [rank_zero] IPU available: False, using: 0 IPUs
2023-04-04 03:51:54,952 [INFO] [rank_zero] HPU available: False, using: 0 HPUs
2023-04-04 03:51:54,960 [INFO] [cuda] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting: 0it [00:00, ?it/s]

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/jovyan/working/src/ice_cube/data_loader.py", line 52, in __getitem__
    x = torch.tensor(x, dtype=torch.float32)
TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.


In [9]:
predictions = (
    torch.cat(results, dim=1).detach().cpu().numpy()
)

In [10]:
predictions

array([[ 9.9333233e-01,  4.9131688e-02, -1.0429241e-01,  2.8534021e+00],
       [-5.6608284e-01, -2.3402302e-01, -7.9043239e-01,  7.8869848e+00],
       [-1.7325106e-01, -9.8453736e-01,  2.5886854e-02,  7.9658882e+01]],
      dtype=float32)

In [31]:
for i in dataloader:
    
    log.info(type(i))
    log.info(type(i.dataset))

    log.info(i.get_example(1).features)
    log.info(i.get_example(1).x.shape)
    log.info(i.get_example(1).x.permute(1, 0, 2, 3, 4, 5))
    log.info(i.get_example(1).x[0])
    log.info(i.get_example(1).y[0])
    log.info(i.get_example(1).z[0])
    log.info(i.get_example(1).time[0])
    log.info(i.get_example(1).charge[0])
    # log.info(i.get_example(1).to_namedtuple())
    break

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f6f8e774f80>
Traceback (most recent call last):
  File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f6f8e774f80><function _MultiProcessingDataLoaderIter.__del__ at 0x7f6f8e774f80><function _MultiProcessingDataLoaderIter.__del__ at 0x7f6f8e774f80>

Traceback (most recent call last):

Traceback (most recent call last):


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/jovyan/working/src/ice_cube/data_loader.py", line 52, in __getitem__
    x = torch.tensor(x, dtype=torch.float32)
TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.


  File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
Traceback (most recent call last):
  File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
      File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()self._shutdown_workers()
    
  File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
self._shutdown_workers()  File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers

          File "/home/sugiyama/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
if w.is_alive():if w.is_alive():
    
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
if w.is_alive():  File "/opt/conda/lib/p